In [ ]:
import numpy as np

#politia es la tabla
#recompesa valor numerico maximice
# funcion de valor para que estado lo bueno q es
# entonrno modelo permite la transicion
 # esta es la cuadricula

#LUIS ANDRES MAMANI BURGOS


#genero mi matriz de 4x4
class Board():
    def __init__(self):
        self.state = np.zeros((4, 4)) # Matriz de 4x4 de ceros

    # Devuelve todos los posibles movimientos válidos (casillas libres)
    def valid_moves(self):
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]

    # Asigna la 'X' o 'O'
    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("¡Movimiento ilegal!")
   # Comprueba si el juego ha terminado
    def is_game_over(self):
        # Comprobar filas y columnas
        for i in range(4):
            if sum(self.state[i, :]) == 4 or sum(self.state[:, i]) == 4:
                return 1
            if sum(self.state[i, :]) == -4 or sum(self.state[:, i]) == -4:
                return -1

        # Comprobar diagonales
        diag_sums = [
            sum([self.state[i, i] for i in range(4)]),
            sum([self.state[i, 3 - i] for i in range(4)]),
        ]
        if 4 in diag_sums:
            return 1
        if -4 in diag_sums:
            return -1

        # Empate
        if len(self.valid_moves()) == 0:
            return 0
        # Seguir jugando
        return None

    def reset(self):
        self.state = np.zeros((4, 4)) # Reiniciar el estado del tablero


In [ ]:
from tqdm import tqdm

# el juego realiza las partidas
class Game():

   #asigna los player
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    #lo ase agarra los dos player para ver quien gane
    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False

            while not game_over: # se ejecuta cada vesque es false
                for player in self.players:# se itera en los dos players

                    action = player.move(self.board)  #cada player llama a su funcion move
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board) #guarda el estado de la tabla
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break

            self.reward() #se la recompensa  ademas quien gana de los 2 players
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins


    def reward(self): # asigna la recompensa
        winner = self.board.is_game_over()
        if winner == 0: # empate
            for player in self.players:
                player.reward(0.5) #empate

        else: # le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)  #gano
                else:
                    player.reward(0)  #perdio

In [ ]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {} # tabla con pares estado -> valor
        self.alpha = alpha         # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp   # probabilidad de explorar

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()

        # exploracion de manera aletoria
        if explore and np.random.uniform(0, 1) < self.prob_exp: #fase de exploracion
            # vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves)) #escoje las posiciones
            return valid_moves[ix] #asta aqui ya que trata de forma aletoria puede encontra algo mejor

        # explotacion
        # vamos a la posición con más valor
        max_value = -1000
        for row, col in valid_moves: #coger los movimientos validos
            next_board = board.state.copy()  #cpioa de la tabla
            next_board[row, col] = self.symbol # asemos los movimientos validos
            next_state = str(next_board.reshape(4*4))

            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state) #vamos a tabla scamos el valor
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col

        return best_row, best_col  #e valor mas grande sabe cual es moviento mas grande o ganar

    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))


    def reward(self, reward): #recompensa
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por tods los estados actualizando su valor en la tabla

        for p in reversed(self.positions):#itera en laposiciones por las q paso
            if self.value_function.get(p) is None:
                self.value_function[p] = 0 # se incia si no esta  en la tabla
            self.value_function[p] += self.alpha * (reward - self.value_function[p])#si esta en al tabla  se suma el lerning reid =hiperparametro
            reward = self.value_function[p] #empiesa del ultimo estado

In [ ]:
#uso mis agentes
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)

game.selfplay(15000)

100%|██████████| 15000/15000 [04:58<00:00, 50.30it/s]


[4794, 3887]

In [ ]:
import pandas as pd

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)# sacamos la funcion de valor de agente uno
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor
0,[ 0. 0. 0. 1. 0. 0. -1. 1. 0. 0. -1. ...,0.999992
1,[ 0. 0. 1. -1. 0. 0. 1. 0. 0. 0. 1. -...,0.999969
2,[ 0. 0. 0. 1. 0. 0. 1. -1. 0. 1. 0. -...,0.999512
3,[ 0. 0. 1. 0. 0. -1. 1. 0. 0. 0. 1. -...,0.999512
4,[ 0. 0. -1. 1. 0. 0. -1. 1. 0. 0. 0. ...,0.999023
...,...,...
124670,[ 0. 1. 0. -1. 1. -1. -1. 1. 0. 1. -1. ...,0.000000
124671,[ 0. 1. 0. 0. 1. -1. -1. 1. 0. 1. -1. ...,0.000000
124672,[ 0. 0. 0. 0. 1. -1. -1. 1. 0. 1. -1. ...,0.000000
124673,[ 0. 1. 1. -1. -1. -1. -1. 1. 1. -1. -1. ...,0.000000


1 donde esta el jugador uno y 0 dondes esta 2  y 0 no hay nadie
estado 1 gna 1el jugador 1
estado 0 pierde

In [ ]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)